In [55]:
#!pip install demoji

In [56]:
#!pip install langdetect

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import nltk
import spacy
import string
import re
import warnings
import demoji


from nltk.corpus import stopwords
from nltk.collocations import *
from collections import Counter
from nltk.stem import porter
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('genesis')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))
stemmer = porter.PorterStemmer()

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

nlp = spacy.load("en_core_web_sm")

nltk.download('punkt')
nltk.download('brown')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\98936\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\98936\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\98936\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\98936\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\98936\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\98936\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [58]:
# Importing the dataset
url = 'https://raw.githubusercontent.com/shahrzadko/NLPDigitalMusic/main/Digital_Music_5.json?token=GHSAT0AAAAAAB6LOL42SKHPCCMIVYPOC4REY7AKE2Q'
df_raw = pd.read_json(url)
full_df = pd.DataFrame.from_records(df_raw['data'])

In [59]:
#creating Sentiment column with overall rating
full_df['Sentiment'] = np.where((full_df['overall'] > 3), 'Positive', 
                          np.where((full_df['overall'] < 3), 'Negative', 'Neutral'))


In [60]:
#showing that the data is not balanced
full_df['Sentiment'].value_counts() 

Positive    158985
Neutral       6792
Negative      4004
Name: Sentiment, dtype: int64

In [61]:
#creating  'NewReview'  column using  'reviewText' and 'summary' columns
cols = ['reviewText','summary']
full_df['NewReview'] = full_df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
full_df['NewReview'] = full_df['NewReview'].astype(str)
full_df['review_length'] = full_df['NewReview'].apply(lambda x: len(x.split()))

In [62]:
music_df_balanced = full_df.drop(columns=['vote','image','reviewTime','style','reviewerName','unixReviewTime','summary','reviewText'])

In [63]:
#dropping rows if the review is not verified
music_df_balanced = music_df_balanced[music_df_balanced.verified == True]

#dropping rows if the reviewer generally gives very negative reviews
df_filtered = music_df_balanced.groupby('reviewerID').filter(lambda x: not((x['overall'].count() >= 10) and (x['overall'].mean() <= 2)))

#dropping rows if the review is too short
#df_filtered = df_filtered[df_filtered['review_length'] >= 6]


In [64]:
#checking the values of Sentiment to creat a sample data
df_filtered['Sentiment'].value_counts() 

Positive    141104
Neutral       5396
Negative      2093
Name: Sentiment, dtype: int64

In [46]:
import pandas as pd
import nltk
from nltk.corpus import wordnet


# Reduce the number of positive sentiment rows to 5000
df_pos = df_filtered[df_filtered['Sentiment'] == 'Positive'].sample(n=10000, random_state=42)

# Split the remaining dataset by sentiment
df_neutral = df_filtered[df_filtered['Sentiment'] == 'Neutral']
df_neg = df_filtered[df_filtered['Sentiment'] == 'Negative']

# Define function for synonym replacement
def synonym_replacement(text):
    words = nltk.word_tokenize(text)
    new_words = []
    for word in words:
        synonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if synonyms:
            new_word = synonyms[0]
        else:
            new_word = word
        new_words.append(new_word)
    new_text = ' '.join(new_words)
    return new_text

# Oversample minority classes using synonym replacement
desired_balance = 0.9
while len(df_neutral) / len(df_pos) < desired_balance or len(df_neg) / len(df_pos) < desired_balance:
    if len(df_neutral) / len(df_pos) < desired_balance:
        sample = df_neutral.sample(n=1)['NewReview'].iloc[0]
        new_review = synonym_replacement(sample)
        df_neutral = df_neutral.append({'NewReview': new_review, 'Sentiment': 'Neutral'}, ignore_index=True)
    if len(df_neg) / len(df_pos) < desired_balance:
        sample = df_neg.sample(n=1)['NewReview'].iloc[0]
        new_review = synonym_replacement(sample)
        df_neg = df_neg.append({'NewReview': new_review, 'Sentiment': 'Negative'}, ignore_index=True)


# Combine oversampled subsets back into a single dataframe
df_balanced = pd.concat([df_pos, df_neutral, df_neg], ignore_index=True)

# Shuffle the dataframe
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)


In [47]:
df_balanced['Sentiment'].value_counts() 

Positive    10000
Neutral      9000
Negative     9000
Name: Sentiment, dtype: int64

In [65]:
#creating balanced data using equal number of each sentiment
# sample_size = min(df_filtered['Sentiment'].value_counts())
# df_balanced = df_filtered.groupby('Sentiment').apply(lambda x: x.sample(sample_size)).reset_index(drop=True)

In [66]:
df_balanced['Sentiment'].value_counts() 

Negative    2093
Neutral     2093
Positive    2093
Name: Sentiment, dtype: int64

In [50]:
df_balanced.head()

,overall,verified,reviewerID,asin,Sentiment,NewReview,review_length
0,5.0,True,A12OSNTDE2U22V,B003DDY3NG,Positive,I love this song by Elvis !! It's sounds great...,15.0
1,3.0,True,A2O1EBPCAGFQ88,B0011Z4Y6E,Neutral,Something a little different from a singer who...,37.0
2,5.0,True,A27D4DN0N153XP,B000W1UAMA,Positive,Love Mary J! Five Stars,5.0
3,NaN,NaN,NaN,NaN,Negative,This song beryllium manner excessively deceler...,NaN
4,NaN,NaN,NaN,NaN,Negative,"iodine make not like this song , evening thoug...",NaN


In [51]:
#drop non-English comments
from langdetect import detect

def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except:
        return False

df_balanced = df_balanced[df_balanced['NewReview'].apply(is_english)]


In [52]:
#checking the values of Sentiment after removing non-English reviews
df_balanced['Sentiment'].value_counts() 

Positive    9462
Neutral     8749
Negative    8157
Name: Sentiment, dtype: int64

In [67]:
df_balanced = df_balanced.drop(columns=['verified','overall','asin','reviewerID'])

In [68]:
df_balanced.head()

,overall,Sentiment,NewReview,review_length
0,1.0,Negative,This is the worst sleep music I have purchased...,39
1,2.0,Negative,Ambrosia Anthology had the same songs plus man...,50
2,2.0,Negative,This was not a version that I expected. Maybe ...,41
3,1.0,Negative,Not what i expected at all. it was not my top ...,24
4,1.0,Negative,I was a fan of Marx years ago but wasn't a fan...,19


In [89]:
df_paper_data = df_balanced

In [23]:
from nltk.stem import *

# Initialize stopwords
STOPWORDS = set(stopwords.words('english'))
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define function for text cleaning
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove currency symbols
    text = re.sub(r'£|\$', '', text)

    # Remove phone numbers
    text = re.sub(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove punctuation and special characters except emojis
    #text = re.sub(r'[^\w\s' + emoji.get_emoji_regexp() + ']', '', text)
    text = demoji.replace(text, '')

    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords and handle negation
    negation_words = ['not', 'no', 'never', 'neither', 'nor']
    words = [word if word.lower() in negation_words else word.lower() for word in words]
    words = [word for word in words if word.lower() not in STOPWORDS]
    words = [f'NOT_{words[i+1]}' if (i < len(words)-1 and words[i].lower() in negation_words and words[i+1] not in string.punctuation) else words[i] for i in range(len(words))]
    
    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]

    # Stem words
    words = [stemmer.stem(word) for word in words]

    # Join words back into text
    cleaned_text = ' '.join(words)
    return cleaned_text


In [24]:
df_balanced['processed_review'] = df_balanced['NewReview'].apply(clean_text)
# from multiprocessing import cpu_count, Pool
# pool = Pool(cpu_count())
# tasks = df_balanced['NewReview'].tolist()
# done = pool.map(clean_text, tasks)  # parallel processing
# df_balanced['processed_review'] = done

In [25]:
df_balanced.head()

,Sentiment,NewReview,review_length,processed_review
0,Positive,Real catchy Four Stars,4.0,real catchi four star
1,Negative,This artist beryllium apparently very popular ...,NaN,thi artist beryllium appar veri popular inch t...
2,Positive,"Cool Song, it brings back memories of when i w...",15.0,"cool song , it bring back memori of when i wa ..."
3,Negative,iodine bargain this for my son . iodine americ...,NaN,iodin bargain thi for my son . iodin americium...
4,Neutral,I have an older Decemberists CD and was happy ...,86.0,i have an older decemberist cd and wa happi th...


In [26]:
df_balanced = df_balanced.drop(columns=['NewReview'])

In [27]:
df_balanced.head()

,Sentiment,review_length,processed_review
0,Positive,4.0,real catchi four star
1,Negative,NaN,thi artist beryllium appar veri popular inch t...
2,Positive,15.0,"cool song , it bring back memori of when i wa ..."
3,Negative,NaN,iodin bargain thi for my son . iodin americium...
4,Neutral,86.0,i have an older decemberist cd and wa happi th...


In [28]:
X = pd.DataFrame(df_balanced['processed_review'])
y = df_balanced['Sentiment']

TfidfVectorizer / Naive Bayes

In [29]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy for TFIDF Vectorizer and Naive Bayes: {acc:.3f}")


Accuracy for TFIDF Vectorizer and Naive Bayes: 0.852


In [31]:
print('confusion matrix and classification report for TFIDF Vectorizer and Naive Bayes')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

confusion matrix and classification report for TFIDF Vectorizer and Naive Bayes
[[2140  217  121]
 [ 343 1920  355]
 [  15  120 2702]]
              precision    recall  f1-score   support

    Negative       0.86      0.86      0.86      2478
     Neutral       0.85      0.73      0.79      2618
    Positive       0.85      0.95      0.90      2837

    accuracy                           0.85      7933
   macro avg       0.85      0.85      0.85      7933
weighted avg       0.85      0.85      0.85      7933



Count Vectorizer / Naive Bayse

In [32]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define CountVectorizer
count_vectorizer = CountVectorizer()

# Define Naive Bayes classifier
nb_classifier = MultinomialNB()

# Combine CountVectorizer and Naive Bayes classifier into a pipeline
pipeline = Pipeline([
    ('vectorizer', count_vectorizer),
    ('classifier', nb_classifier)
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy Accuracy for Count Vectorizer and Naive Bayes: {accuracy}")


Accuracy Accuracy for Count Vectorizer and Naive Bayes: 0.8290684482541283


In [33]:
print('confusion matrix and classification report for Count Vectorizer and Naive Bayes')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

confusion matrix and classification report for Count Vectorizer and Naive Bayes
[[2047  257  174]
 [ 380 1815  423]
 [   9  113 2715]]
              precision    recall  f1-score   support

    Negative       0.84      0.83      0.83      2478
     Neutral       0.83      0.69      0.76      2618
    Positive       0.82      0.96      0.88      2837

    accuracy                           0.83      7933
   macro avg       0.83      0.83      0.82      7933
weighted avg       0.83      0.83      0.83      7933



#### Logistic Regression

In [34]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
model = make_pipeline(
    CountVectorizer(),
    StandardScaler(with_mean=False),
    LogisticRegression(random_state=42)
)

# Fit the model to the training data
model.fit(X_train['processed_review'], y_train)

# Predict the sentiment of the test data
y_pred = model.predict(X_test['processed_review'])

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Logistic Regression model: {accuracy}")

Accuracy of Logistic Regression model: 0.8870540779024328


c:\Users\98936\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
print('confusion matrix and classification report of Logistic Regression model')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

confusion matrix and classification report of Logistic Regression model
[[2333   93   52]
 [ 122 2241  255]
 [  72  302 2463]]
              precision    recall  f1-score   support

    Negative       0.92      0.94      0.93      2478
     Neutral       0.85      0.86      0.85      2618
    Positive       0.89      0.87      0.88      2837

    accuracy                           0.89      7933
   macro avg       0.89      0.89      0.89      7933
weighted avg       0.89      0.89      0.89      7933



#### Gradient Boosting

In [36]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('gb', GradientBoostingClassifier())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy of Gradient Boosting: {acc:.3f}")


Accuracy of Gradient Boosting: 0.841


In [37]:
print('confusion matrix and classification report of Gradient Boosting')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

confusion matrix and classification report of Gradient Boosting
[[2109  212  157]
 [ 349 1905  364]
 [  28  150 2659]]
              precision    recall  f1-score   support

    Negative       0.85      0.85      0.85      2478
     Neutral       0.84      0.73      0.78      2618
    Positive       0.84      0.94      0.88      2837

    accuracy                           0.84      7933
   macro avg       0.84      0.84      0.84      7933
weighted avg       0.84      0.84      0.84      7933



SVM

In [38]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy of SVM before tuning: {acc:.3f}")


Accuracy of SVM before tuning: 0.919


In [39]:
print('confusion matrix and classification report of SVM before tuning')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

confusion matrix and classification report of SVM before tuning
[[2295  125   58]
 [  72 2306  240]
 [  16  134 2687]]
              precision    recall  f1-score   support

    Negative       0.96      0.93      0.94      2478
     Neutral       0.90      0.88      0.89      2618
    Positive       0.90      0.95      0.92      2837

    accuracy                           0.92      7933
   macro avg       0.92      0.92      0.92      7933
weighted avg       0.92      0.92      0.92      7933



In [40]:
# # Split the data into training and testing sets with stratified sampling
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 

# # Define the pipeline
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer()),
#     ('svm', SVC())
# ])


# # Define the hyperparameter grid
# param_grid = {
#     'svm__C': [0.1, 1, 10, 100,200],
#     'svm__kernel': ['linear', 'rbf', 'sigmoid','precomputed','poly'],
#     'svm__gamma': ['scale', 'auto','float','array-like']
# }


# # Define the GridSearchCV object
# grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)

# # Fit the pipeline on the training data
# grid_search.fit(X_train['processed_review'], y_train)

# # Print the best hyperparameters
# print("Best parameters: ", grid_search.best_params_)

# # Predict on the test data using the best estimator
# y_pred = grid_search.best_estimator_.predict(X_test['processed_review'])

# # Calculate the accuracy
# acc = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {acc:.3f}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\98936\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
300 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\98936\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\98936\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\98936\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 177, in fit
    raise TypeError("Sparse precomputed kernels are not supported

Best parameters:  {'svm__C': 100, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Accuracy: 0.928


In [41]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(C= 100, gamma= 'scale', kernel='rbf'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")

Accuracy: 0.928


In [42]:
print('confusion matrix and classification report of SVM after tuning')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

confusion matrix and classification report of SVM after tuning
[[2349   88   41]
 [  70 2341  207]
 [  18  151 2668]]
              precision    recall  f1-score   support

    Negative       0.96      0.95      0.96      2478
     Neutral       0.91      0.89      0.90      2618
    Positive       0.91      0.94      0.93      2837

    accuracy                           0.93      7933
   macro avg       0.93      0.93      0.93      7933
weighted avg       0.93      0.93      0.93      7933



In [43]:
from sklearn.calibration import CalibratedClassifierCV

# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(C= 100, gamma= 'scale', kernel='rbf', probability=True)) # set probability to True
])

# Wrap the pipeline in a calibrated classifier
calibrated_pipeline = CalibratedClassifierCV(pipeline, method='sigmoid', cv=5) # you can choose different calibration methods

# Fit the pipeline on the training data
calibrated_pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = calibrated_pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


Accuracy: 0.927


In [44]:
print('confusion matrix and classification report of SVM after calibration')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

confusion matrix and classification report of SVM after calibration
[[2339   89   50]
 [  69 2346  203]
 [  16  154 2667]]
              precision    recall  f1-score   support

    Negative       0.96      0.94      0.95      2478
     Neutral       0.91      0.90      0.90      2618
    Positive       0.91      0.94      0.93      2837

    accuracy                           0.93      7933
   macro avg       0.93      0.93      0.93      7933
weighted avg       0.93      0.93      0.93      7933



In [115]:
def extract_opinion_words(review):
    words = word_tokenize(review.lower())
    pos_tags = nltk.pos_tag(words)
    opinion_words = []
    for word, tag in pos_tags:
        if tag.startswith('JJ') or tag.startswith('VB'):
            opinion_words.append(word)
    return opinion_words
# Initialize stopwords
STOPWORDS = set(stopwords.words('english'))
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define function for text cleaning
def clean_text(text):
    # # Convert to lowercase
    # text = text.lower()

    # # Remove URLs
    # text = re.sub(r'http\S+', '', text)

    # # Remove currency symbols
    # text = re.sub(r'£|\$', '', text)

    # # Remove phone numbers
    # text = re.sub(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', '', text)

    # # Remove digits from NewReview
    # df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'\d+', '', x))

    # # Remove email addresses
    # text = re.sub(r'\S+@\S+', '', text)

    # # Remove punctuation and special characters except emojis
    # text = re.sub(r'[^\w\s'  + ']', '', text)

    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # # Remove stopwords and handle negation
    # negation_words = ['not', 'no', 'never', 'neither', 'nor']
    # words = [word if word.lower() in negation_words else word.lower() for word in words]
    # words = [word for word in words if word.lower() not in STOPWORDS]
    # words = [f'NOT_{words[i+1]}' if (i < len(words)-1 and words[i].lower() in negation_words and words[i+1] not in string.punctuation) else words[i] for i in range(len(words))]
    
    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]

    # Stem words
    words = [stemmer.stem(word) for word in words]

    # Join words back into text
    cleaned_text = ' '.join(words)
    return cleaned_text
df_paper_data['processed_review'] = df_paper_data['NewReview'].apply(clean_text)
df_paper_data['processed_review'] = df_paper_data['processed_review'].apply(extract_opinion_words)
df_paper_data["processed_review"] = df_paper_data["processed_review"].apply(lambda x: " ".join(x))
X_paper = pd.DataFrame(df_paper_data['processed_review'])
y_paper = df_paper_data['Sentiment']
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X_paper, y_paper, test_size=0.3, stratify=y_paper, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy for TFIDF Vectorizer and Naive Bayes: {acc:.3f}")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

C:\Users\98936\AppData\Local\Temp\ipykernel_10648\4051919555.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paper_data['processed_review'] = df_paper_data['NewReview'].apply(clean_text)


Accuracy for TFIDF Vectorizer and Naive Bayes: 0.620
[[352 171  48]
 [131 387  91]
 [ 68 162 357]]
              precision    recall  f1-score   support

    Negative       0.64      0.62      0.63       571
     Neutral       0.54      0.64      0.58       609
    Positive       0.72      0.61      0.66       587

    accuracy                           0.62      1767
   macro avg       0.63      0.62      0.62      1767
weighted avg       0.63      0.62      0.62      1767



C:\Users\98936\AppData\Local\Temp\ipykernel_10648\4051919555.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paper_data['processed_review'] = df_paper_data['processed_review'].apply(extract_opinion_words)
C:\Users\98936\AppData\Local\Temp\ipykernel_10648\4051919555.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paper_data["processed_review"] = df_paper_data["processed_review"].apply(lambda x: " ".join(x))
